# Chapter 15: Context Managers

Master resource management with context managers and the 'with' statement



### What are Context Managers? (Slide 36)


<p><strong>Context Manager</strong> - Objects that manage resources with automatic setup and cleanup</p>
<p><strong>The 'with' Statement:</strong></p>
<ul>
<li>Ensures resources are properly acquired and released</li>
<li>Guarantees cleanup even if exceptions occur</li>
<li>Makes code cleaner and safer</li>
</ul>
<p><strong>Common Use Cases:</strong></p>
<ul>
<li>File handling - auto-close files</li>
<li>Database connections - auto-commit/rollback</li>
<li>Locks - auto-release</li>
<li>Timing code execution</li>
<li>Temporary state changes</li>
</ul>
<p><strong>Protocol:</strong> Implement <code>__enter__</code> and <code>__exit__</code> methods</p>


### The 'with' Statement (Slide 37)


In [1]:
# Without context manager
file = open('data.txt', 'r')
try:
    data = file.read()
    print(data)
finally:
    file.close()  # Must remember to close!

# With context manager - much cleaner!
with open('data.txt', 'r') as file:
    data = file.read()
    print(data)
# File automatically closed here, even if exception!

# Multiple context managers
with open('input.txt', 'r') as infile, \
     open('output.txt', 'w') as outfile:
    for line in infile:
        outfile.write(line.upper())
# Both files automatically closed

# Exception handling still works
try:
    with open('data.txt', 'r') as f:
        data = f.read()
        raise ValueError("Something wrong")
except ValueError:
    pass  # File still closed properly!


10
20
30
40
55
60
77
80
99
100

10
20
30
40
55
60
77
80
99
100



> **Note:** with ensures cleanup in all cases


### Creating Custom Context Manager (Slide 38)


In [2]:
# Custom context manager class
class FileManager:
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode
        self.file = None

    def __enter__(self):
        print(f"Opening {self.filename}")
        self.file = open(self.filename, self.mode)
        return self.file  # Returned to 'as' variable

    def __exit__(self, exc_type, exc_value, traceback):
        print(f"Closing {self.filename}")
        if self.file:
            self.file.close()
        # Return False to propagate exceptions
        # Return True to suppress exceptions
        return False

# Use custom context manager
with FileManager('test.txt', 'w') as f:
    f.write('Hello, World!')
# Output:
# Opening test.txt
# Closing test.txt


Opening test.txt
Closing test.txt


> **Note:** __enter__ runs on entry, __exit__ on exit


### Exception Handling in Context Managers (Slide 39)


In [3]:
class ErrorHandler:
    def __enter__(self):
        print("Entering context")
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        print(f"Exiting context")

        if exc_type is not None:
            print(f"Exception occurred: {exc_type.__name__}")
            print(f"Message: {exc_value}")
            # Return True to suppress exception
            return True

        return False

# Exception is suppressed
with ErrorHandler():
    print("Inside context")
    raise ValueError("Something went wrong")
    print("This won't print")

print("Program continues")  # This runs!

# Output:
# Entering context
# Inside context
# Exiting context
# Exception occurred: ValueError
# Message: Something went wrong
# Program continues


Entering context
Inside context
Exiting context
Exception occurred: ValueError
Message: Something went wrong
Program continues


> **Note:** Return True from __exit__ to suppress exceptions


### contextlib.contextmanager Decorator (Slide 40)


In [4]:
from contextlib import contextmanager

# Simpler way to create context managers
@contextmanager
def file_manager(filename, mode):
    print(f"Opening {filename}")
    file = open(filename, mode)
    try:
        yield file  # Code runs here
    finally:
        print(f"Closing {filename}")
        file.close()

# Use like any context manager
with file_manager('test.txt', 'w') as f:
    f.write('Hello from decorator!')

# Timing example
@contextmanager
def timer(name):
    import time
    start = time.time()
    yield
    end = time.time()
    print(f"{name} took {end-start:.4f}s")

with timer("My operation"):
    # Do some work
    sum([i**2 for i in range(100000)])
# Output: My operation took 0.0234s


Opening test.txt
Closing test.txt
My operation took 0.0051s


> **Note:** @contextmanager simplifies context manager creation


### Real-World - Database Transactions (Slide 41)


In [5]:
from contextlib import contextmanager

class Database:
    def __init__(self, db_name):
        self.db_name = db_name
        self.connection = None

    def connect(self):
        print(f"Connecting to {self.db_name}")
        self.connection = f"Connection to {self.db_name}"

    def disconnect(self):
        print(f"Disconnecting from {self.db_name}")
        self.connection = None

    @contextmanager
    def transaction(self):
        """Auto-commit or rollback"""
        print("BEGIN TRANSACTION")
        try:
            yield self
            print("COMMIT")
        except Exception as e:
            print(f"ROLLBACK due to: {e}")
            raise

db = Database("mydb")
db.connect()

# Successful transaction
with db.transaction():
    print("INSERT data...")
    print("UPDATE records...")
# Auto-commits

# Failed transaction
try:
    with db.transaction():
        print("DELETE records...")
        raise Exception("Error!")
except:
    pass  # Auto-rollback happened


Connecting to mydb
BEGIN TRANSACTION
INSERT data...
UPDATE records...
COMMIT
BEGIN TRANSACTION
DELETE records...
ROLLBACK due to: Error!


> **Note:** Context managers ensure proper transaction handling


### Real-World - Temporary Directory (Slide 42)


In [6]:
import os
import tempfile
from contextlib import contextmanager

@contextmanager
def temporary_directory():
    """Create and cleanup temp directory"""
    temp_dir = tempfile.mkdtemp()
    print(f"Created temp dir: {temp_dir}")
    try:
        yield temp_dir
    finally:
        # Cleanup
        import shutil
        shutil.rmtree(temp_dir)
        print(f"Cleaned up: {temp_dir}")

# Use temporary directory
with temporary_directory() as tmpdir:
    # Create files in temp directory
    file_path = os.path.join(tmpdir, 'data.txt')
    with open(file_path, 'w') as f:
        f.write('Temporary data')

    print(f"Working in: {tmpdir}")
    print(f"File exists: {os.path.exists(file_path)}")

# Directory and files are gone!
print(f"Cleaned up: {not os.path.exists(tmpdir)}")


Created temp dir: C:\Users\tkpar\AppData\Local\Temp\tmprxi99c1v
Working in: C:\Users\tkpar\AppData\Local\Temp\tmprxi99c1v
File exists: True
Cleaned up: C:\Users\tkpar\AppData\Local\Temp\tmprxi99c1v
Cleaned up: True


> **Note:** Great for temporary resources that need cleanup


### Suppressing Exceptions (Slide 43)


In [7]:
from contextlib import suppress

# Without suppress
try:
    os.remove('file_that_might_not_exist.txt')
except FileNotFoundError:
    pass  # Ignore if file doesn't exist

# With suppress - much cleaner!
with suppress(FileNotFoundError):
    os.remove('file_that_might_not_exist.txt')

# Suppress multiple exception types
with suppress(FileNotFoundError, PermissionError):
    os.remove('protected_file.txt')

# Real-world example
def cleanup_files(filenames):
    for filename in filenames:
        with suppress(FileNotFoundError):
            os.remove(filename)
    print("Cleanup complete")

cleanup_files(['temp1.txt', 'temp2.txt', 'temp3.txt'])
# Won't crash if files don't exist


Cleanup complete


> **Note:** suppress() for cleaner exception ignoring


### Redirect stdout/stderr (Slide 44)


In [8]:
from contextlib import redirect_stdout, redirect_stderr
import io
import time

# Capture stdout to string
output = io.StringIO()
with redirect_stdout(output):
    print("This goes to string, not console")
    print("Multiple lines")

captured = output.getvalue()
print(f"Captured output: {captured}")

# Redirect to file
with open('output.log', 'w') as f:
    with redirect_stdout(f):
        print("This goes to file")
        print("Logged at:", time.time())

# Suppress output completely
import os
with redirect_stdout(open(os.devnull, 'w')):
    print("This is silenced")
    # Useful for noisy third-party libraries

# Redirect stderr
errors = io.StringIO()
with redirect_stderr(errors):
    import sys
    sys.stderr.write("Error message")

print(f"Errors: {errors.getvalue()}")


Captured output: This goes to string, not console
Multiple lines

Errors: Error message


> **Note:** Useful for testing and logging


### Context Manager Best Practices (Slide 45)


<p><strong>When to Use Context Managers:</strong></p>
<ul>
<li>Resource management (files, connections, locks)</li>
<li>Setup/teardown operations</li>
<li>Temporary state changes</li>
<li>Transaction handling</li>
<li>Timing and profiling</li>
</ul>
<p><strong>Do:</strong></p>
<ul>
<li>Use <code>@contextmanager</code> for simple cases</li>
<li>Always implement cleanup in finally or __exit__</li>
<li>Make context managers reusable</li>
<li>Use built-in context managers when available</li>
<li>Document what resources are managed</li>
</ul>
<p><strong>Don't:</strong></p>
<ul>
<li>Forget to handle exceptions in __exit__</li>
<li>Suppress exceptions silently without good reason</li>
<li>Make context managers too complex</li>
<li>Forget cleanup can fail too</li>
</ul>
